# Importing Library

In [2]:
import pandas as pd
from datetime import datetime,date
data = pd.read_excel('/home/rohan/Desktop/test.xlsx')
data

,Time_differ,Break_time,Lunch_time
0,00:20:18,00:00:00,01:40:40
1,00:20:18,00:30:18,01:40:30
2,04:20:18,00:00:00,02:40:30
3,00:20:18,00:20:18,03:40:30
4,00:00:00,00:30:18,04:40:30
5,00:20:18,00:20:18,05:40:30


In [15]:
data[['Time_differ','Break_time']]x

,Time_differ,Break_time
0,2019-01-22 00:20:18,2019-01-22 00:00:00
1,2019-01-22 00:20:18,2019-01-22 00:30:18
2,2019-01-22 04:20:18,2019-01-22 00:00:00
3,2019-01-22 00:20:18,2019-01-22 00:20:18
4,2019-01-22 00:00:00,2019-01-22 00:30:18
5,2019-01-22 00:20:18,2019-01-22 00:20:18


In [4]:
data.dtypes

Time_differ    object
Break_time     object
Lunch_time     object
dtype: object

In [12]:
#data = data.applymap(lambda x:datetime.combine(date.today(),x))
data

,Time_differ,Break_time,Lunch_time
0,2019-01-22 00:20:18,2019-01-22 00:00:00,2019-01-22 01:40:40
1,2019-01-22 00:20:18,2019-01-22 00:30:18,2019-01-22 01:40:30
2,2019-01-22 04:20:18,2019-01-22 00:00:00,2019-01-22 02:40:30
3,2019-01-22 00:20:18,2019-01-22 00:20:18,2019-01-22 03:40:30
4,2019-01-22 00:00:00,2019-01-22 00:30:18,2019-01-22 04:40:30
5,2019-01-22 00:20:18,2019-01-22 00:20:18,2019-01-22 05:40:30


In [13]:
data.dtypes

Time_differ    datetime64[ns]
Break_time     datetime64[ns]
Lunch_time     datetime64[ns]
dtype: object

In [14]:
data.Time_differ - data.Break_time

0            00:20:18
1   -1 days +23:50:00
2            04:20:18
3            00:00:00
4   -1 days +23:29:42
5            00:00:00
dtype: timedelta64[ns]

# ----------------------------------------CaseStudy---------------------------------------------

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns

#____________________________________________________

from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn import metrics
import pandas_profiling
from sklearn.neighbors import KNeighborsClassifier 

# Datasets

###### convert all dates to Datetime from object type

# ------------------------------------------Customer_details------------------------------------

- Gender has 99 / 4.2% missing values Missing --> RandomFill
- StreeID has 39 / 1.7% zeros Zeros --> Randomfill
- DOB - object --> convert dtype

In [321]:
cust = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/customers.txt',sep=";")
cust.head(2)

,CustomerID,Gender,DOB,District,ZIP,StreeID
0,908289,F,24/01/1957,5,3660,42555
1,55837,M,31/01/1948,1,2620,24910


In [324]:
# checking if the primary is repetitive
cust.CustomerID.nunique() == cust.shape[0]

True

In [202]:
cust.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,2356.0,534904.313243,382112.919352,3407.0,130850.25,616509.0,852283.5,1167808.0
District,2356.0,3.455008,2.045992,1.0,1.00,5.0,5.0,8.0
ZIP,2356.0,3432.750424,1272.641443,1070.0,2640.00,3530.0,3700.0,9800.0
StreeID,2356.0,36592.521647,17942.070858,0.0,25083.00,38524.5,43773.5,146620.0


- Fixing StreeiD by filling it with Random values bw (20000,50000)
- We are doing it randomly coz only 2% values are 0 so we can do it.

In [431]:
## using replace to replace a particular value in a DF column with random values.
#cust.replace('StrrID',{0:np.arange(1,39)})

In [7]:
cust.loc[cust.StreeID == 0,['StreeID']]  = cust[cust.StreeID == 0].StreeID.apply(lambda x:np.random.randint(20000,50000))
#cust[cust0,cust[cust.StreeID == 0].StreeID.apply(lambda x:np.random.randint(20000,50000)))


- Limit arg provides the flexibility to fill first n values with a speified_value

In [10]:
# Filliing half M and half F using Limit argument in Fill_Na
#cust.Gender.fillna('F',limit= 50,inplace= True)
#cust.Gender.fillna('M',limit= 50,inplace= True)
cust.Gender.isnull().sum()

0

In [12]:
cust.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,2356.0,534904.313243,382112.919352,3407.0,130850.25,616509.0,852283.50,1167808.0
District,2356.0,3.455008,2.045992,1.0,1.00,5.0,5.00,8.0
ZIP,2356.0,3432.750424,1272.641443,1070.0,2640.00,3530.0,3700.00,9800.0
StreeID,2356.0,37159.372666,17344.839571,2251.0,25367.50,38676.5,43818.25,146620.0


# ------------------------------------------subscription-----------------------------------------------

###### A lot of variables are highly corelated,so we can consider removing a few


- FormulaDiscount has 334 / 2.2% missing values Missing --> Drop it || ~ Total discount
- FormulaID has 334 / 2.2% missing values Missing 
- GrossFormulaPrice has 334 / 2.2% missing values Missing --> Drop it || ~ NFP--> Total Price
- NetFormulaPrice has 334 / 2.2% missing values Missing --> Drop it || ~ Total Price
- NetNewspaperPrice has 334 / 2.2% missing values Missing --> Drop it || ~ Total Price
- ProductDiscount has 334 / 2.2% missing values Missing --> Drop it || ~ Total discount
- TotalCredit has 4877 / 31.5% missing values Missing 
- TotalDiscount has 334 / 2.2% missing values Missing
- TotalPrice has 334 / 2.2% missing values Missing
- Unnamed: 4 has 15459 / 99.9% missing values Missing --> Drop it
- nbrNewspaper - Drop it || ~ Totalprice
- -------------------------------------------------------------------------------

- Pattern - denotes delivery pattern in a week



##### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- Gross formula price(GFP), Net Formula price(NFP)


- GFP <> NFP || product_discount <> Formula discount || Total credit


- N.Newspaper price(NNP) = price of newspaper



- Total Price = f(NNP,Nbrnewspaper)
- Total Price = f(NFP/GFP, Total credit)

- We would be dropping **GFP|NFP|NNP** and keeping **Total_price|total credit|total discount**

In [435]:
#pandas_profiling.ProfileReport(subs)

In [282]:
subs = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/subscriptions.txt',sep="\t")
subs.head(2)

,SubscriptionID,CustomerID,ProductID,Pattern,Unnamed: 4,StartDate,EndDate,NbrNewspapers,NbrStart,RenewalDate,...,PaymentDate,FormulaID,GrossFormulaPrice,NetFormulaPrice,NetNewspaperPrice,ProductDiscount,FormulaDiscount,TotalDiscount,TotalPrice,TotalCredit
0,1000024.0,1150045.0,8.0,1111110.0,=VLOOKUP(D2;$E$15462:$F$15473;2;0),18/01/2010,17/04/2010,76.0,25,17/03/2010,...,27/01/2010,8552,79,79,1.039.474,0,0,0,79,0
1,1000082.0,1150046.0,8.0,1111110.0,NaN,14/01/2010,12/03/2010,50.0,25,3/03/2010,...,11/02/2010,10017,519.737,29.6,0.592,0,223.737,223.737,29.6,0


In [207]:
subs.shape

(15472, 22)

- Dropping the columns 

In [260]:
subs.columns

Index(['SubscriptionID', 'CustomerID', 'ProductID', 'Pattern', 'Unnamed: 4',
       'StartDate', 'EndDate', 'NbrNewspapers', 'NbrStart', 'RenewalDate',
       'PaymentType', 'PaymentStatus', 'PaymentDate', 'FormulaID',
       'GrossFormulaPrice', 'NetFormulaPrice', 'NetNewspaperPrice',
       'ProductDiscount', 'FormulaDiscount', 'TotalDiscount', 'TotalPrice',
       'TotalCredit'],
      dtype='object')

- Dropping the 2.2%(334) values from the whole dataset coz multiple columns had **'NAN'** for that 

- Here it can be seen wherever TotalPrice is NAN, All others(which have NANs) are NANs


- Also dropping Unnamed:4 column

In [291]:
#subs.drop(index=subs[subs.TotalPrice.isnull()].index,inplace= True)
#subs.drop(columns= 'Unnamed: 4',inplace= True)

In [292]:
subs.head(2)

,SubscriptionID,CustomerID,ProductID,Pattern,StartDate,EndDate,NbrNewspapers,NbrStart,RenewalDate,PaymentType,...,PaymentDate,FormulaID,GrossFormulaPrice,NetFormulaPrice,NetNewspaperPrice,ProductDiscount,FormulaDiscount,TotalDiscount,TotalPrice,TotalCredit
0,1000024.0,1150045.0,8.0,1111110.0,18/01/2010,17/04/2010,76.0,25,17/03/2010,BT,...,27/01/2010,8552,79,79,1.039.474,0,0,0,79,0
1,1000082.0,1150046.0,8.0,1111110.0,14/01/2010,12/03/2010,50.0,25,3/03/2010,BT,...,11/02/2010,10017,519.737,29.6,0.592,0,223.737,223.737,29.6,0


- Dropping the columns that create **multucollinearity**

- Total credit has 30% values missing 

-  we know that **Total price = NFP - Total Credit**

In [316]:
subs[['TotalPrice','NetFormulaPrice','TotalCredit']].iloc[[4,11,16]]

,TotalPrice,NetFormulaPrice,TotalCredit
4,246.8,249,-2.2
12,0,1.039.474,NaN
17,0,0.754237,NaN


In [317]:
#subs_f = subs.drop(columns= ['FormulaDiscount','GrossFormulaPrice','NetNewspaperPrice','ProductDiscount', 'FormulaDiscount','TotalCredit'])

In [318]:
subs_f.head(2)

,SubscriptionID,CustomerID,ProductID,Pattern,StartDate,EndDate,NbrNewspapers,NbrStart,RenewalDate,PaymentType,PaymentStatus,PaymentDate,FormulaID,NetFormulaPrice,TotalDiscount,TotalPrice
0,1000024.0,1150045.0,8.0,1111110.0,18/01/2010,17/04/2010,76.0,25,17/03/2010,BT,Paid,27/01/2010,8552,79,0,79
1,1000082.0,1150046.0,8.0,1111110.0,14/01/2010,12/03/2010,50.0,25,3/03/2010,BT,Paid,11/02/2010,10017,29.6,223.737,29.6


In [320]:
# pandas_profiling.ProfileReport(subs_f)

In [340]:
# checking if the primary is repetitive
print('unique subscription ID:',subs_f.SubscriptionID.nunique(),'|| total obs in dataset:',subs_f.shape[0])
subs_f.SubscriptionID.nunique() == subs_f.shape[0]

unique subscription ID: 15138 || total obs in dataset: 15138


True

In [341]:
a = subs_f.groupby(subs_f.SubscriptionID).count()

print('CustomerID:',a.CustomerID.unique(),'|| ProductID:',a.ProductID.unique())

CustomerID: [1] || ProductID: [1]


# --------------------------------------------------Delivery-------------------------------------------

In [437]:
#pandas_profiling.ProfileReport(Del)

- DeliveryClass has 3388 / 12.8% missing values Missing --> treated using "fillna('bfill')"
- DeliveryContext has 15148 / 57.2% missing values Missing - Defined only for abnormal(ABN) case --> Dropped

In [345]:
# delivery type - [DR - delivery reruting]&[Main Delivery]

Del = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/delivery.txt',sep=";")
Del.head(2)

,SubscriptionID,DeliveryType,DeliveryClass,DeliveryContext,StartDate,EndDate
0,1000024,MD,NOR,NaN,18/01/2010,17/04/2010
1,1000082,MD,ABN,NaN,12/02/2010,13/02/2010


In [346]:
Del.DeliveryClass.value_counts()

NOR    21996
ABN     1110
Name: DeliveryClass, dtype: int64

In [138]:
Del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26494 entries, 0 to 26493
Data columns (total 6 columns):
SubscriptionID     26494 non-null int64
DeliveryType       26494 non-null object
DeliveryClass      23106 non-null object
DeliveryContext    11346 non-null object
StartDate          26494 non-null object
EndDate            26494 non-null object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


#### preparing the data 

In [347]:
# As can be seen the variable DeliveryConetxt is very much dependent on DeliveryClass
# (out of the 15k DeliveryConetxt value,14K values are for DeliveryClass 'NOR')

# We would be dropping the DeliveryConetxt column

Del[Del.DeliveryClass == 'NOR'].DeliveryContext.isnull().sum()


14125

In [353]:
#Dropping the Deliveryconetxt column - too many missing values & also Multicollinearity
#Del.drop(columns='DeliveryContext',inplace=True)

In [354]:
Del.head(2)

,SubscriptionID,DeliveryType,DeliveryClass,StartDate,EndDate
0,1000024,MD,NOR,18/01/2010,17/04/2010
1,1000082,MD,ABN,12/02/2010,13/02/2010


#### Using KNN for predicting the missing 12% values for DeliveryClass var is not useful coz we have very less columns to train the model

In [507]:
#Del = pd.get_dummies(Del,prefix='Deliverytype_',columns=['DeliveryType'],drop_first= True)

#________Converting Datatypes to Datetime from string class____

# Del['StartDate'] = pd.to_datetime(Del['StartDate'])
# Del['EndDate'] = pd.to_datetime(Del['EndDate'])

#### Based on bins(subscription_id) of data, we can create synthetic labels - Not working 

#### Moving to "bfill" for filling the values for the categroical value

In [367]:
#Del['Decile'] = pd.qcut(Del.index,10,labels=False)

In [375]:
Del.head(2)

,SubscriptionID,DeliveryType,DeliveryClass,StartDate,EndDate
0,1000024,MD,NOR,18/01/2010,17/04/2010
1,1000082,MD,ABN,12/02/2010,13/02/2010


In [166]:
#v = Del.loc[Del.Decile == 0,'DeliveryClass'].fillna(value ='NOR',limit = int(Del.loc[Del.Decile==0,'DeliveryClass'].isnull().sum()*0.8))
#v = Del.loc[Del.Decile == 0,'DeliveryClass'].fillna(value ='ABN')
#v.isnull().sum()
#Del.DeliveryClass = v
#Del.DeliveryClass.isnull().sum()

In [365]:
# v = []
# for x in range(10):
#     v3 = Del.loc[Del.Decile == x,'DeliveryClass'].fillna('NOR',limit = int(Del[Del.Decile==x].shape[0]*0.8))
#     v3 = Del.loc[Del.Decile == x,'DeliveryClass'].fillna('ABN')
#     v3 = v1.tolist()
#     v.extend(v3)
    
# Del['DeliveryClass'] = pd.Series(v3)

In [355]:
Del.DeliveryClass.fillna(method = 'bfill',inplace=True)

In [356]:
Del.DeliveryClass.value_counts()

NOR    25176
ABN     1318
Name: DeliveryClass, dtype: int64

In [357]:
Del.head(2)

,SubscriptionID,DeliveryType,DeliveryClass,StartDate,EndDate
0,1000024,MD,NOR,18/01/2010,17/04/2010
1,1000082,MD,ABN,12/02/2010,13/02/2010


In [363]:
# checking if the primary key is repetitive
print('unique subscription ID:',Del.SubscriptionID.nunique(),'|| total obs in dataset:',Del.shape[0])
Del.SubscriptionID.nunique() == Del.shape[0]


unique subscription ID: 15137 || total obs in dataset: 26494


False

# ------------------------------------Formula----------------------------------------------------

- Duration in month

In [365]:
Formula = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/formula.txt',sep=";")
Formula.head(2)

,FormulaID,FormulaCode,FormulaType,Duration
0,10011,2051,CAM,12
1,10013,2051,CAM,12


In [368]:
# checking if the primary key is repetitive
print('unique Formula ID:',Formula.FormulaID.nunique(),'|| total obs in dataset:',Formula.shape[0])
Formula.FormulaID.nunique() == Formula.shape[0]

unique Formula ID: 997 || total obs in dataset: 997


True

# -----------------------------------------------Credit-------------------------------------------------

In [ ]:
#pandas_profiling.ProfileReport(credit)

- nbrNewspapers -  no of newspaper on credit  

- --------------------------------------------------------------------------------
- Missing_value

- NbrNewspapers has 532 / 27.4% missing values Missing - only 7 unique values --> converted to 0 [0 rs]
- ProcessingDate has 38 / 2.0% missing values Missing --> filled using fillna() 
- Dataset has 350 duplicate rows Warning --> to fix


In [370]:
credit = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/credit.txt',sep=";")
credit.head(2)

,SubscriptionID,ActionType,ProcessingDate,CreditSource,Amount,NbrNewspapers
0,1002057,EN,18/05/2009,COM,1.0400,1.0
1,1002089,EN,06/10/2010,COM,0.8783,1.0


#### preparing the data

- since we have 900 "1" and 500 "0" and around "500" NAN, we can convert NAN to 0 coz:
      - They should actually be 0
      - The data would get balanced by "1" & "0" [1000 vs 1000] 
     

In [412]:
credit.groupby(['NbrNewspapers']).count()

,SubscriptionID,ActionType,ProcessingDate,CreditSource,Amount
NbrNewspapers,,,,,
0.0,487,487,453,487,487
1.0,914,914,914,914,914
5.0,1,1,0,1,1
14.0,1,1,1,1,0
15.0,1,1,1,1,0
22.0,2,2,0,2,2
28.0,1,1,1,1,0
40.0,1,1,0,1,1


In [371]:
## All NAN values are for "CC" ActionType
credit.loc[credit.NbrNewspapers.isnull(),['ActionType','NbrNewspapers']].head(2)

,ActionType,NbrNewspapers
54,CC,NaN
92,CC,NaN


In [375]:
### Equating all NAN values to 0

#credit.loc[credit.NbrNewspapers.isnull(),'NbrNewspapers'] = 0
credit.NbrNewspapers.isnull().sum() #-- No Null Values

0

- Filling the missing "Processing_date" column

In [380]:
#credit.ProcessingDate.fillna(method= 'ffill',inplace=True)
credit.head()

,SubscriptionID,ActionType,ProcessingDate,CreditSource,Amount,NbrNewspapers
0,1002057,EN,18/05/2009,COM,1.0400,1.0
1,1002089,EN,06/10/2010,COM,0.8783,1.0
2,1002089,EN,22/12/2010,COM,0.8783,1.0
3,1002089,EN,23/12/2010,COM,0.8783,1.0
4,1002089,EN,24/12/2010,COM,0.8783,1.0


In [381]:
# checking if the primary key is repetitive
print('unique SubscriptionID:',credit.SubscriptionID.nunique(),'|| total obs in dataset:',credit.shape[0])
credit.SubscriptionID.nunique() == credit.shape[0]

unique SubscriptionID: 1112 || total obs in dataset: 1940


False

# ---------------------------------------------Complains---------------------------------------------

In [ ]:
#pandas_profiling.ProfileReport(complains)

- Complain_type -  (1-9)
- Solution_type - (1-4)
- complain_date -  object type



###### Missing_Value
- ProductID has 414 / 10.0% missing values Missing
- SolutionType has 1092 / 26.5% missing values Missing
- Feedback_type - FeedbackType has 2502 / 60.7% missing values

In [382]:
complains = pd.read_csv('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/1. NEWS PUBLISHING COMPANY-CLASSIFICATION-ATTRITION/2 - NPC Retention Data/complaints.txt',sep=";")
complains.head(2)

,ComplaintID,CustomerID,ProductID,ComplaintDate,ComplaintType,SolutionType,FeedbackType
0,181932,3464,8.0,14/10/2010,7,2.0,NaN
1,168698,3464,8.0,19/07/2010,1,2.0,3.0


#### Fixing the data

- **complaint_type** and **Feedback_type** are very similar columns moreover Feedback_type has 60% NAN so we are dropping it

In [384]:
#Dropping the column 
# complains.drop(columns= 'FeedbackType',inplace=True)

In [385]:
complains.head(2)

,ComplaintID,CustomerID,ProductID,ComplaintDate,ComplaintType,SolutionType
0,181932,3464,8.0,14/10/2010,7,2.0
1,168698,3464,8.0,19/07/2010,1,2.0


- Using **KNN** for predicting the 1000~ values for solutiontype(dependent on Complaint_type)

### --------------------------------------------KNN for unseen data----------------------

In [386]:
## we can see here that soltiontype has total 4125 obs, out of which 10xy are NAN
## out of remaining 3k, solutiontype 1,2 comprise 2k both of which are pretty much dependent on
## on complainttype = 1,2

x = complains.groupby(['SolutionType','ComplaintType']).count()
x.loc[:2]
#x.xs(1) # can also be used like this - cross section 

ComplaintID  CustomerID  ProductID  ComplaintDate
SolutionType ComplaintType                                                   
1.0          1                     1027        1027        911           1027
             2                        9           9          8              9
             3                       25          25         20             25
             4                        2           2          2              2
             6                        1           1          1              1
             7                       35          35         31             35
             8                        1           1          1              1
             9                      308         308        286            308
2.0          1                     1005        1005        841           1005
             2                        4           4          4              4
             3                       33          33         28             33
             7                       34          34         30             34
             9                       16          16         10             16

In [387]:
## Out of all null values for solutiontype we have the division based on complainttype

x = complains[complains.SolutionType.isnull()]
x.groupby(['ComplaintType']).count()

,ComplaintID,CustomerID,ProductID,ComplaintDate,SolutionType
ComplaintType,,,,,
1,821,821,819,821,0
2,50,50,50,50,0
3,19,19,19,19,0
4,4,4,4,4,0
5,16,16,16,16,0
6,3,3,3,3,0
7,87,87,87,87,0
8,6,6,6,6,0
9,86,86,86,86,0


In [388]:
complains.head(2)

,ComplaintID,CustomerID,ProductID,ComplaintDate,ComplaintType,SolutionType
0,181932,3464,8.0,14/10/2010,7,2.0
1,168698,3464,8.0,19/07/2010,1,2.0


In [389]:
complain_M = complains[['ComplaintType','ComplaintDate','SolutionType']]

- converting **datetime to integer** so to use it for modelling

In [390]:
complain_M.head(3)

,ComplaintType,ComplaintDate,SolutionType
0,7,14/10/2010,2.0
1,1,19/07/2010,2.0
2,1,11/06/2010,2.0


In [391]:
complain_M.dtypes

ComplaintType      int64
ComplaintDate     object
SolutionType     float64
dtype: object

In [392]:
complain_M['ComplaintDate'] = pd.to_datetime(complain_M.ComplaintDate)

/home/rohan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [393]:
complain_M.dtypes

ComplaintType             int64
ComplaintDate    datetime64[ns]
SolutionType            float64
dtype: object

In [394]:
complain_M.ComplaintDate = complain_M.ComplaintDate - pd.to_datetime('1900-01-01')
complain_M.ComplaintDate  = complain_M.ComplaintDate.dt.days.astype('int')
complain_M.ComplaintDate.head()

/home/rohan/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


0    40463
1    40376
2    40486
3    39926
4    39943
Name: ComplaintDate, dtype: int64

In [395]:
complain_M.head(2)

,ComplaintType,ComplaintDate,SolutionType
0,7,40463,2.0
1,1,40376,2.0


In [396]:
train = complain_M[complain_M.SolutionType.notnull()]

## This is not the testing data but the data on which the model is to be applied
predict_test = complain_M[complain_M.SolutionType.isnull()]

print('train_data:',train.shape,'|| predict_data:',predict_test.shape)

train_data: (3033, 3) || predict_data: (1092, 3)


In [397]:
x_train,x_test,y_train,y_test = train_test_split(train[train.columns.difference(['SolutionType'])],train['SolutionType'],test_size = 0.2)

In [398]:
print('x_train:',x_train.shape,'|| x_test:',x_test.shape,'|| y_train:',y_train.shape,'|| y_test:',y_test.shape)


x_train: (2426, 2) || x_test: (607, 2) || y_train: (2426,) || y_test: (607,)


In [422]:
knn_grid  = GridSearchCV(KNeighborsClassifier(),{'n_neighbors':[3,5,7,9],'leaf_size':[10,30,40,50,60] },scoring='accuracy',cv=5)
knn_grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [3, 5, 7, 9], 'leaf_size': [10, 30, 40, 50, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [423]:
knn_grid.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [424]:
x =  pd.Series(test_reault)
x.value_counts()

1.0    298
2.0    217
3.0     79
4.0     13
dtype: int64

In [425]:
test_reault = knn_grid.best_estimator_.predict(x_test)

In [428]:
metrics.accuracy_score(y_test,test_reault)

0.528830313014827

#### Using this KNN model and predicting the unseen data

In [429]:
predict_test.columns

Index(['ComplaintType', 'ComplaintDate', 'SolutionType'], dtype='object')

In [430]:
u = predict_test[['ComplaintDate','ComplaintType']]
u.ComplaintType.value_counts()

1    821
7     87
9     86
2     50
3     19
5     16
8      6
4      4
6      3
Name: ComplaintType, dtype: int64

In [431]:
pred_data = knn_grid.best_estimator_.predict(u)
pred_data

array([1., 2., 1., ..., 1., 1., 3.])

In [432]:
Target = pd.Series(pred_data)
Target.value_counts()

1.0    603
2.0    278
4.0    185
3.0     26
dtype: int64

#### Also tried other methods but the accuracy was too less ~ 50-60%, So sticking with KNN

In [433]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier

# # rf_cls = GridSearchCV(estimator= RandomForestClassifier(oob_score= True),cv = 5,scoring= 'accuracy',
# #                       param_grid={'n_estimators':[50,100,200,300],'max_depth':[2,5,10,20]}
# #                      )
# bag_cls = GridSearchCV(estimator= BaggingClassifier(oob_score= True),cv = 5,scoring= 'accuracy',
#                       param_grid={'n_estimators':[100,200,300]},n_jobs = -1
#                      )


# bag_cls.fit(x_train,y_train)

#------------------------------------------------------------------------------------

# bag_test = bag_cls.best_estimator_.predict(x_test)

# bag_cls.best_params_

# metrics.accuracy_score(y_test,bag_test)

# x = bag_cls.predict(predict_test[['ComplaintType', 'ComplaintDate']])

# x

# x = pd.Series(x)
# x.value_counts()

#### Recreating data

In [434]:
np.shape(pred_data)

(1092,)

In [435]:
predict_test.shape

(1092, 3)

In [436]:
predict_test['SolutionType'] = pred_data

/home/rohan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [437]:
predict_test.SolutionType.value_counts()

1.0    603
2.0    278
4.0    185
3.0     26
Name: SolutionType, dtype: int64

####  Final Dataset

In [438]:
com = pd.concat([train,predict_test])

In [439]:
com.head(3)

,ComplaintType,ComplaintDate,SolutionType
0,7,40463,2.0
1,1,40376,2.0
2,1,40486,2.0


- Dropping the integer type Complaintdate & replacing by the actual one

In [442]:
#com.drop(columns= 'ComplaintDate',inplace=True)
complains_f = com.join([complains.ComplaintID,complains.CustomerID,complains.ProductID,complains.ComplaintDate])

In [443]:
complains_f = complains_f[['ComplaintID','CustomerID','ProductID','ComplaintDate', 'ComplaintType','SolutionType']]
complains_f.sort_index(inplace=True)

- **Comparing the final & the raw dataset**

In [444]:
# Final Dataset
complains_f.head(2)

,ComplaintID,CustomerID,ProductID,ComplaintDate,ComplaintType,SolutionType
0,181932,3464,8.0,14/10/2010,7,2.0
1,168698,3464,8.0,19/07/2010,1,2.0


In [446]:
complains.head()

,ComplaintID,CustomerID,ProductID,ComplaintDate,ComplaintType,SolutionType
0,181932,3464,8.0,14/10/2010,7,2.0
1,168698,3464,8.0,19/07/2010,1,2.0
2,163694,4107,8.0,11/06/2010,1,2.0
3,111664,4870,8.0,25/04/2009,1,3.0
4,139482,4870,8.0,05/12/2009,1,3.0


In [492]:
a = complains_f.ComplaintID.duplicated(keep= False)
b  = complains_f.loc[a,['ComplaintID']]
b.sort_values('ComplaintID').head(4)

,ComplaintID
3652,52454
2101,52454
3420,55018
727,55018


In [451]:
# checking if the primary key is repetitive
print('unique ComplaintID:',complains_f.ComplaintID.nunique(),'|| total obs in dataset:',complains.shape[0])
complains.ComplaintID.nunique() == complains.shape[0]

unique ComplaintID: 4107 || total obs in dataset: 4125


False

### --------------------------------------------------------------------------------------------------------------------------

# Master Dataset